In [19]:
import cv2
import os
import numpy as np
import glob
import math
import scipy
from scipy.spatial import distance
from scipy import signal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn import metrics

In [20]:
##Lấy danh sách mẫu mống mắt
cwd = os.getcwd()
for dir_path, dir_names, file_names in os.walk(cwd):
    for f in file_names:
        list_file = open("list_file.txt","a+")
        temp=str(os.path.join(dir_path,f)+'\n')
        if '.bmp' in temp:
            list_file.write(temp)
list_file = open("list_file.txt","r")
images_train=[]
images_test=[]
ID_images_train=[]
ID_images_test=[]
if list_file.mode == 'r':
    data =list_file.readlines()
    for i in data[300:350]:
        if i[-6:-5]=='1':
            ID_images_test.append(i[-10:-5])
            images_test.append(cv2.imread(i[:-1]))
        else:
            ID_images_train.append(i[-10:-5])
            images_train.append(cv2.imread(i[:-1]))

In [21]:
print(ID_images_test)

['061_1', '062_1', '063_1', '064_1', '065_1', '066_1', '067_1', '068_1', '069_1', '070_1']


In [22]:
print(ID_images_train)

['061_2', '061_3', '061_4', '061_5', '062_2', '062_3', '062_4', '062_5', '063_2', '063_3', '063_4', '063_5', '064_2', '064_3', '064_4', '064_5', '065_2', '065_3', '065_4', '065_5', '066_2', '066_3', '066_4', '066_5', '067_2', '067_3', '067_4', '067_5', '068_2', '068_3', '068_4', '068_5', '069_2', '069_3', '069_4', '069_5', '070_2', '070_3', '070_4', '070_5']


In [23]:
def ImageEnhancement(normalized):
    enhanced=[]
    for res in normalized:
        res = res.astype(np.uint8)
        im=cv2.equalizeHist(res)
        enhanced.append(im)
    return enhanced

In [24]:
#modulating function as defined in paper
def m(x ,y, f):
    val = np.cos(2*np.pi*f*math.sqrt(x **2 + y**2))
    return val
#spatial filter as defined in paper
def gabor(x, y, dx, dy, f):
    gb = (1/(2*math.pi*dx*dy))*np.exp(-0.5*(x**2 / dx**2 + y**2 / dy**2)) * m(x, y, f)
    return gb

#function to calculate spatial filter over 8x8 blocks
def spatial(f,dx,dy):
    sfilter=np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            sfilter[i,j]=gabor((-4+j),(-4+i),dx,dy,f)
    return sfilter

def get_vec(convolvedtrain1,convolvedtrain2):
    feature_vec=[]
    for i in range(6):
            for j in range(64):
                #Run 8 by 8 filtered block iteratively over the entire image
                start_height = i*8
                end_height = start_height+8
                start_wid = j*8
                end_wid = start_wid+8
                grid1 = convolvedtrain1[start_height:end_height, start_wid:end_wid]
                grid2 = convolvedtrain2[start_height:end_height, start_wid:end_wid]

                # Channel 1
                absolute = np.absolute(grid1)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

                # Channel 2
                absolute = np.absolute(grid2)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

    return feature_vec

def FeatureExtraction(enhanced):
    con1=[]
    con2=[]
    #get spatial filters
    filter1=spatial(0.67,3,1.5)
    filter2=spatial(0.67,4,1.5) 
    
    feature_vector=[]
    
    for i in range(len(enhanced)):
        img=enhanced[i]
        #define a 48x512 region over which the filters are applied
        img_roi=img[:48,:]
        
        filtered1=scipy.signal.convolve2d(img_roi,filter1,mode='same')
        filtered2=scipy.signal.convolve2d(img_roi,filter2,mode='same')
        
        con1.append(filtered1)
        con2.append(filtered2)
        fv=get_vec(filtered1,filtered2)
        feature_vector.append(fv)
    return feature_vector #each feature vector has a dimension of 1536



In [25]:
images_train=[ImageEnhancement(i) for i in images_train]
train=[FeatureExtraction(i) for i in images_train]

In [26]:
images_test=[ImageEnhancement(i) for i in images_test]
test=[FeatureExtraction(i) for i in images_test]

In [27]:
feature_vector_test=test
feature_vector_train=train

In [28]:
def score(feature_vector_train,feature_vector_test):
    score=0
    for i in range(0,1536):
        if np.isnan(feature_vector_train[i])==True and np.isnan(feature_vector_test[i])==True:
            score+=0
        elif np.isnan(feature_vector_train[i])==True:
            score+=feature_vector_test[i]
        elif np.isnan(feature_vector_test[i])==True:
            score+=feature_vector_train[i]
        else:
            score+=abs(feature_vector_train[i]-feature_vector_test[i])
    return score

In [29]:
a=[[sum([score(feature_vector_train[i][j],feature_vector_test[k][j]) for j in range(0,48)]) for i in range(0,40)] for k in range(0,10)]

In [30]:
a

[[1014.5983211338295,
  419.1936151557601,
  380.2936591660975,
  406.55302545733144,
  834.1013879040336,
  788.6103391919157,
  771.9357709148544,
  810.6051649685011,
  786.9962437600636,
  822.8587746828103,
  826.5055848478951,
  836.7576192850781,
  805.2363888315347,
  794.5822456726225,
  806.8491833574105,
  823.0679871432349,
  778.9584589674388,
  724.7884174832379,
  741.2428422743693,
  761.6657259658665,
  982.2952736696367,
  935.2634974316843,
  956.384925674538,
  956.3351969808978,
  867.9190057158236,
  855.5894738174952,
  872.7335602909357,
  876.7721385549182,
  925.8159827080127,
  874.8091848094303,
  925.8818538750386,
  883.039223224599,
  983.1846207769058,
  865.8299267145935,
  888.7442031516952,
  878.5196657003396,
  723.8090942927256,
  657.3630958002094,
  715.8209869374004,
  737.6599098515986],
 [913.6132466357195,
  799.5142652607781,
  804.8224599569803,
  850.3549459365742,
  248.96834078590115,
  567.584606913578,
  384.47129745169883,
  422.75985

In [31]:
result=[ID_images_train[i.index(min(i))][0:3] for i in a]

In [32]:
print(result)

['061', '062', '063', '064', '065', '066', '067', '068', '069', '070']


In [33]:
ID_images_test=[i[0:3] for i in ID_images_test ]

In [34]:
print(ID_images_test)

['061', '062', '063', '064', '065', '066', '067', '068', '069', '070']


In [35]:
from scipy.spatial import distance

In [36]:
1-distance.hamming(result,ID_images_test)

1.0